In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store, fetch_predictions
import pandas as pd
from datetime import datetime, timedelta, timezone


def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc))
    fetch_data_from = current_date - timedelta(days=(365+days)) 
    fetch_data_to = current_date - timedelta(days=365)
    print(fetch_data_from, fetch_data_to)
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    return df[cond]

In [4]:
ts_data = fetch_days_data(180)

2023-08-18 23:35:17.696020+00:00 2024-02-14 23:35:17.696020+00:00
2025-02-13 18:35:17,699 INFO: Initializing external client
2025-02-13 18:35:17,701 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 18:35:19,210 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.90s) 


In [5]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [6]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()
pipeline.fit(features, targets)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.153639 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171646
[LightGBM] [Info] Number of data points in the train set: 29120, number of used features: 675
[LightGBM] [Info] Start training from score 16.297012


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001EB62B88FE0>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [7]:
from sklearn.metrics import mean_absolute_error
predictions = pipeline.predict(features)

In [8]:
test_mae = mean_absolute_error(targets, predictions)
print(f"{test_mae:.4f}")

3.1804


In [9]:
from src.inference import load_metrics_from_registry 

metric = load_metrics_from_registry()

2025-02-13 18:37:43,999 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 18:37:44,013 INFO: Initializing external client
2025-02-13 18:37:44,014 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 18:37:45,147 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [10]:
metric

{'test_mae': 1.935489834251996}

In [11]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

2025-02-13 18:37:57,122 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 18:37:57,135 INFO: Initializing external client
2025-02-13 18:37:57,135 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 18:37:58,164 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [12]:
import joblib  
import src.config
# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl']

In [13]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [15]:
model_registry = project.get_model_registry()
model_registry

ModelRegistry(project: 'mkzia')

In [16]:
modelv2 = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor V2",
    input_example=features.sample(),
    model_schema=model_schema,
)

In [17]:
modelv2.save('C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/315730 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/2903 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/51418 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1211551/models/taxi_demand_predictor_next_hour/2


Model(name: 'taxi_demand_predictor_next_hour', version: 2)

In [19]:
from src.inference import load_model_from_registry
model = load_model_from_registry(-1)

2025-02-13 18:44:34,141 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 18:44:34,153 INFO: Initializing external client
2025-02-13 18:44:34,154 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 18:44:35,300 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1211551/modelregistries/1211551/models/taxi_demand_predictor_next_hour_-1). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":360000,"errorMsg":"No model found for provided name and version."}', error code: 360000, error msg: No model found for provided name and version., user msg: 

In [20]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [25]:
models

[Model(name: 'taxi_demand_predictor_next_hour', version: 1),
 Model(name: 'taxi_demand_predictor_next_hour', version: 2)]

In [26]:
max(models, key=lambda model: model.version)


Model(name: 'taxi_demand_predictor_next_hour', version: 2)

In [28]:
load_metrics_from_registry()

2025-02-13 18:58:48,415 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 18:58:48,421 INFO: Initializing external client
2025-02-13 18:58:48,423 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 18:58:49,603 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


{'test_mae': 3.180406299167993}